In [1]:
import requests,re,time,numpy as np,pickle,asyncio,concurrent.futures,datetime
# Make this notebook wiiiiiiiiide
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

This parser works with just a few locations, constructing an ITA-style matrix of price vs start/end dates

In [2]:
# Setup relevant variables
ZIP = '60615'
# AGENCYLIST = [('ET','E11509'), #oak park,
#              #('ET','E11548'), #bolingbrook
#              #('ET','E115CH'), #elmwood
#              #('ET','E11516'), #skokie
#              #('ET','E11553'), #Nn Lincolnwood 53
#             ]
AGENCYLIST = None
RADIUS = '40'

PICKUP = datetime.date(2017, 11, 28)
DROPOFF = PICKUP+datetime.timedelta(days=21)
FIXEDLENGTH = True
VARYSTART = 1
VARYEND = 2
TIME = '11:00 AM'

In [3]:
%run backend_withdb.ipynb

In [4]:
if not AGENCYLIST:
    (agencylist,_) = getagencylist((PICKUP,DROPOFF))
    AGENCYLIST = [(x['bcode'],x['code']) for x in agencylist.values()]

In [5]:
AGENCYLIST

[('ET', 'E11586'),
 ('ET', 'E11576'),
 ('AV', 'CHIC05'),
 ('BG', 'CHIC02'),
 ('ET', 'E115DN'),
 ('BG', 'CHIC08'),
 ('ET', 'E11530'),
 ('ET', 'E115DA'),
 ('AV', 'CHIX01'),
 ('BG', 'CHIX01'),
 ('ET', 'E115CE'),
 ('ET', 'E115CD'),
 ('ET', 'E11555'),
 ('ET', 'E115KB'),
 ('AV', 'CHIC01'),
 ('BG', 'CHIC09'),
 ('AL', 'CHIC72'),
 ('ET', 'E115CA'),
 ('ET', 'E115C6'),
 ('ET', 'E115CU'),
 ('ET', 'E11557'),
 ('ET', 'E115C7'),
 ('AV', 'MDWT01'),
 ('BG', 'MDWT02'),
 ('ET', 'E115CQ'),
 ('ET', 'MDWT61'),
 ('AL', 'MDWT71'),
 ('ET', 'E115C4'),
 ('AV', 'CHIC12'),
 ('BG', 'CHIC12'),
 ('AV', 'CHIS08'),
 ('BG', 'CHIS04'),
 ('ET', 'E115MT'),
 ('ET', 'E11503'),
 ('ET', 'E115CF'),
 ('ET', 'E11578'),
 ('ET', 'E11575'),
 ('ET', 'E1159D'),
 ('ET', 'E115FX'),
 ('ET', 'E11510'),
 ('AV', 'CHIC04'),
 ('BG', 'CHIC01'),
 ('BG', 'CHIC07'),
 ('AV', 'CHIC07'),
 ('AV', 'CHIS04'),
 ('BG', 'CHIS02'),
 ('ET', 'E115C9'),
 ('AV', 'CHIW12'),
 ('BG', 'CHIW09'),
 ('ET', 'E11509'),
 ('ET', 'E11588'),
 ('ET', 'E11581'),
 ('ET', 'E11

In [6]:
populateagenciesdb((PICKUP,DROPOFF))

In [7]:
runparser(agencylist=AGENCYLIST, threads=1);

102 ND |  11/27/2017 12/18/2017 35 enterprise E115CF Sat Howard Orloff ET 8.76
103 ND |  11/28/2017 12/19/2017 35 enterprise E115CF Sat Howard Orloff ET 8.76
104 ND |  11/29/2017 12/20/2017 35 enterprise E115CF Sat Howard Orloff ET 8.76
114 ND |  11/27/2017 12/18/2017 39 enterprise E115FX Nn Fox Ford ET 9.58
115 ND |  11/28/2017 12/19/2017 39 enterprise E115FX Nn Fox Ford ET 9.58
116 ND |  11/29/2017 12/20/2017 39 enterprise E115FX Nn Fox Ford ET 9.58
279 ND |  11/27/2017 12/18/2017 94 enterprise E115P1 Packer Branch Elmhurst ET 19.34
280 ND |  11/28/2017 12/19/2017 94 enterprise E115P1 Packer Branch Elmhurst ET 19.34
281 ND |  11/29/2017 12/20/2017 94 enterprise E115P1 Packer Branch Elmhurst ET 19.34
321 ND |  11/27/2017 12/18/2017 108 enterprise E115PD Packer Br Downers Grove ET 21.30
322 ND |  11/28/2017 12/19/2017 108 enterprise E115PD Packer Br Downers Grove ET 21.30
323 ND |  11/29/2017 12/20/2017 108 enterprise E115PD Packer Br Downers Grove ET 21.30
543 ND |  11/27/2017 12/18/2

In [ ]:
res = session.query(Agency).filter(Agency.ag_id == 'E11509').first()
print(res)
res.prices

In [ ]:
session.query(Price.car_econ, Price.pickup, Agency.ag_dist, Agency.ag_name).join(Agency).filter(Price.car_econ.isnot(None)).order_by(Price.car_comp).count()

In [12]:
res = session.query(Price.car_int, Price.pickup, Agency.ag_dist, Agency.ag_brand, Agency.ag_name).join(Agency).filter(Price.car_econ.isnot(None)).order_by(Price.car_int).all()

In [18]:
for i,v in enumerate(res):
    print('{:4d} | {:5.2f} | {:5s} | {:6.2f} | {:10s} | {:40s}'
              .format(i,v[0],v[1],v[2],v[3],v[4]))

   0 | 375.33 | 11/29/2017 |  21.31 | enterprise | Aa South Downers Grove                  
   1 | 375.33 | 11/28/2017 |  21.31 | enterprise | Aa South Downers Grove                  
   2 | 383.19 | 11/27/2017 |  18.03 | budget     | Oak Brook Il                            
   3 | 383.19 | 11/29/2017 |  18.03 | budget     | Oak Brook Il                            
   4 | 383.19 | 11/28/2017 |  18.03 | budget     | Oak Brook Il                            
   5 | 383.19 | 11/27/2017 |  20.30 | budget     | Lombard                                 
   6 | 383.19 | 11/29/2017 |  20.30 | budget     | Lombard                                 
   7 | 383.19 | 11/28/2017 |  20.30 | budget     | Lombard                                 
   8 | 386.84 | 11/27/2017 |  21.35 | budget     | Downers Grove Il                        
   9 | 386.84 | 11/29/2017 |  21.35 | budget     | Downers Grove Il                        
  10 | 386.84 | 11/28/2017 |  21.35 | budget     | Downers Grove Il             

In [13]:
import pprint
pprint.pprint(res)

[(375.33000000000004,
  '11/29/2017',
  21.31,
  'enterprise',
  'Aa South Downers Grove'),
 (375.33000000000004,
  '11/28/2017',
  21.31,
  'enterprise',
  'Aa South Downers Grove'),
 (383.19, '11/27/2017', 18.03, 'budget', 'Oak Brook Il'),
 (383.19, '11/29/2017', 18.03, 'budget', 'Oak Brook Il'),
 (383.19, '11/28/2017', 18.03, 'budget', 'Oak Brook Il'),
 (383.19, '11/27/2017', 20.3, 'budget', 'Lombard'),
 (383.19, '11/29/2017', 20.3, 'budget', 'Lombard'),
 (383.19, '11/28/2017', 20.3, 'budget', 'Lombard'),
 (386.84000000000003, '11/27/2017', 21.35, 'budget', 'Downers Grove Il'),
 (386.84000000000003, '11/29/2017', 21.35, 'budget', 'Downers Grove Il'),
 (386.84000000000003, '11/28/2017', 21.35, 'budget', 'Downers Grove Il'),
 (386.84000000000003, '11/27/2017', 23.57, 'budget', 'Addison'),
 (386.84000000000003, '11/29/2017', 23.57, 'budget', 'Addison'),
 (386.84000000000003, '11/28/2017', 23.57, 'budget', 'Addison'),
 (386.84000000000003, '11/27/2017', 24.47, 'budget', 'The Hyatt Of Li

In [ ]:
car1 = 'Economy Car'
car2 = 'Compact Car'
car3 = 'Intermediate Car'
car4 = 'Intermediate SUV'
desiredcar = car2
#desiredsort = 'time'
desiredsort = desiredcar
desiredbrand = 'enterprise'

# also, can optimize into some single query...buuuut it works as is
rvals = np.array([x for x in results.values() if x is not None and desiredcar in x and desiredsort in x])
orderpreferred = np.array([x[desiredsort] for x in rvals]).argsort()
orderprice = np.array([x[desiredcar] for x in rvals]).argsort()
order = [x for x in orderpreferred if x in orderprice[:60]]

In [ ]:
orderedresults = rvals[order]
print(desiredcar,car2,car3)
for k in orderedresults:
    #print(k['num'])
    if 'time' in k:
        print('{:s} | {:5.2f} | {:3d} | {:6.2f} | {:6.2f} | {:6.2f} | {:10s} | {}'
              .format(k['start']+'-'+k['end'],float(k['dist']),k['time']//60,k[desiredcar],k[car2],k[car3],str(k['brand']),k['name']))
    else:
        print('{:s} | {:5.2f} | {:3d} | {:6.2f} | {:6.2f} | {:6.2f} | {:10s} | {}'
              .format(k['start']+'-'+k['end'],float(k['dist']),0,k[desiredcar],k[car2],k[car3],str(k['brand']),k['name']))